In [71]:
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport
from tqdm import tqdm
import json, time
import sys, os
import requests
import queries
import string
import copy

In [2]:
with open(os.path.expanduser("~/.canvas-token")) as f:
    headers = {"Authorization": "Bearer "+f.read().strip()}
    
DATA_DIR = os.path.expanduser("~/temp/sched2ics-data")

In [3]:
transport = RequestsHTTPTransport(url="https://canvas.cmu.edu/api/graphql", headers=headers)
client = Client(transport=transport, fetch_schema_from_transport=True)

In [4]:
toplevel = client.execute(queries.syllabus_registry())

# Pull out the valid semesters (ignore archive for now)
sems = [x for x in toplevel["course"]["modulesConnection"]["nodes"] if x["name"].endswith(")")]
sis_ids = {}
for sem in sems:
    semname = sem["name"]
    print(semname)
    semid = semname.split("(")[-1][:-1]
    ids = [x["content"]["url"].split(":")[-1] for x in sem["moduleItems"]]
    assert all(semid in x for x in ids)
    assert semid not in sis_ids
    sis_ids[semid] = ids

Fall 2022 (F22)
Summer 2022 (N22)
Summer 2022 (M22)
Spring 2022 (S22)
Fall 2021 (F21)
Summer 2021 (N21)
Summer 2021 (M21)
Spring 2021 (S21)
Fall 2020 (F20)
Summer 2020 (N20)
Summer 2020 (M20)
Spring 2020 (S20)
Fall 2019 (F19)
Summer 2019 (N19)
Summer 2019 (M19)


In [14]:
def get_all_items(courseid, moduleid):
    out = []
    url = f"https://canvas.cmu.edu/api/v1/courses/{courseid}/modules/{moduleid}/items"
    while True:
        res = requests.get(url, headers=headers)
        out += res.json()
        n = [x for x in res.headers["Link"].split(",") if "rel=\"next\"" in x]
        if len(n) == 0: break
        url = n[0][1:-13]
        
    return out

def get_syllabi(sis_id, verbose=False):
    dat = client.execute(queries.get_syllabi(sis_id))
    courseid = dat["course"]["_id"]
    dat = dat["course"]["modulesConnection"]["nodes"]
   
    dat = [x for x in dat if x["name"] == "Available Syllabi"]
    assert len(dat) == 1
    dat = dat[0]
    
    moduleid = dat["_id"]
    dat = dat["moduleItems"]
    
    if verbose: print("Course Module", courseid, moduleid)
    
    ## Weird bug with GQL API necessitates using REST here to get title fields
    dat2 = get_all_items(courseid, moduleid)
    
    courses = []
    for c in dat2:
        if ":" not in c["title"] or "-" not in c["title"]:
            if "." in c["title"] and "-" in c["title"] and (c["title"].split(".")[-1] in ["pdf", "docx"]):
                course_num = c["title"].split("-")[0]
                course_sec = c["title"].split("-")[1].split(".")[0]
                course_name = course_num
            else:
                print("Warning: Invalid title", c["title"])
                continue
        else:
            course_num = c["title"].split("-")[0]
            course_sec = c["title"].split("-")[1].split(":")[0]
            course_name = c["title"][c["title"].index(":")+1:].strip()
        page_type = c["type"]
        assert page_type in ["Page", "File"]
        page_url = c["html_url"]
        
        if page_type == "Page":
            page_id = c["page_url"]
            html = requests.get(f"https://canvas.cmu.edu/api/v1/courses/{courseid}/pages/{page_id}",
                 headers=headers).json()["body"]
            
            if len(html) < 50:
                print("Warning: blank or short page found", course_num, course_sec, page_url)
                continue
        
        if verbose: print(course_num, course_sec, page_url)
        courses.append((course_num, course_sec, course_name, page_type, page_url))
    
    return courses

In [17]:
syllabi_by_sem = {}

print(list(sis_ids.keys()))
for sem, ids in sis_ids.items():
    syllabi = {}
    for sis_id in tqdm(ids):
        s = get_syllabi(sis_id, verbose=False)
        for num, sec, name, ftype, url in s:
            if num not in syllabi:
                syllabi[num] = []
            syllabi[num].append((num, sec, name, ftype, url))
    
    syllabi_by_sem[sem] = syllabi

['F22', 'N22', 'M22', 'S22', 'F21', 'N21', 'M21', 'S21', 'F20', 'N20', 'M20', 'S20', 'F19', 'N19', 'M19']


 10%|████████████                                                                                                           | 6/59 [00:17<03:09,  3.58s/it]

 12%|██████████████                                                                                                         | 7/59 [00:42<09:20, 10.78s/it]

 37%|████████████████████████████████████████████                                                                          | 22/59 [01:53<04:37,  7.50s/it]

 39%|██████████████████████████████████████████████                                                                        | 23/59 [02:02<04:42,  7.85s/it]

...

 95%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏     | 58/61 [03:52<00:09,  3.20s/it]

 97%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏   | 59/61 [03:58<00:07,  3.84s/it]

 20%|███████████████████████▏                                                                                              | 12/61 [00:06<00:27,  1.77it/s]

 21%|█████████████████████████▏                                                                                            | 13/61 [00:08<00:50,  1.04s/it]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 61/61 [00:45<00:00,  1.33it/s]


In [150]:
# We love lazy-loading
# i'm writing this at 3:15am realizing this would all
# be so much cleaner if i did it in a functional language
course_info_cache = {}
def get_courses(sem):
    assert len(sem) == 3
    if sem not in course_info_cache:
        with open(os.path.join(DATA_DIR, "courses_"+sem[0]+"20"+sem[1:]+".json")) as f:
            course_info_cache[sem] = json.load(f)
    
    return course_info_cache[sem]

def get_sections(sem, course):
    c = get_courses(sem).get(course, [])
    c = [x for x in c if ("Pittsburgh" in x["campus"])]
    return c

def _clean_sec_name(sec):
    sec = sec.strip()
    if sec == "Lec":
        return "1"
    if sec.startswith("Lec ") and sec.split(" ")[-1].strip().isnumeric():
        return sec.split(" ")[-1].strip()
    
    return sec

def get_sections_clean(sem, course):
    c2 = get_courses(sem).get(course, [])
    c = [x for x in c2 if ("Pittsburgh" in x["campus"])]
    other_campuses = [_clean_sec_name(x["section"]) for x in c2 if ("Pittsburgh" not in x["campus"])]
    c = copy.deepcopy(c)
    for x in c:
        x["section"] = _clean_sec_name(x["section"])
        
    return c, other_campuses

def _parse_instructors(dat):
    dat = dat["instructors"].strip()
    if dat == "Instructor TBA" or dat == "TBA":
        return set([])
    else:
        return set([a.strip() for a in dat.split(",")])

def get_unique_sections(sem, course):
    sections = get_sections(sem, course)
    if len(dat) == 0:
        return []

    s = [x["section"] for x in sections]
    s2 = [x for x in sections if
            (_clean_sec_name(x["section"]).isnumeric()) or
            (x["section"] in string.ascii_uppercase) or 
            (x["section"][0] in string.ascii_uppercase and x["section"][1] in string.digits) or
            (x["section"][0] in string.ascii_uppercase and x["section"][1] in string.ascii_uppercase)]
    
    if len(s2) == 0:
        return []
    
    # very basic heuristic; if there exists at least one section with all instructors
    # listed (and other sections have some subset of those instructors) we assume
    # they are all the same instructors and just SOC being weird; if there is any
    # amount of disjoint-ness then we 
    instructor_sets = [_parse_instructors(b) for b in s2]
    largest_set = sorted(instructor_sets, key=lambda x: len(x))[-1]
    all_same_instructors = all(x.issubset(largest_set) for x in instructor_sets)
    
    
    if all_same_instructors:
        x = copy.deepcopy(s2[0])
        if len(largest_set) == 0:
            x["instructors"] = "Instructor TBA"
        else:
            x["instructors"] = ", ".join(sorted(list(largest_set)))
        return [x]
    else:
        return [x for x in s2]


In [ ]:
"""
S = "F21"
for course in get_courses(S):
    dat = get_sections(S, course)
    if len(dat) == 0:
        continue
    s = [x["section"] for x in dat]
    s2 = [(x in string.ascii_uppercase) or (x[0] in string.ascii_uppercase and x[1] in string.digits) for x in s]
    
    if not any(s2):
        print(course, s)
"""

In [ ]:
get_syllabi("syllabus-registry-F22-CS", verbose=False)

In [19]:
syllabi_by_sem.keys()

dict_keys(['F22', 'N22', 'M22', 'S22', 'F21', 'N21', 'M21', 'S21', 'F20', 'N20', 'M20', 'S20', 'F19', 'N19', 'M19'])

In [21]:
with open("temp.json", "w+") as f:
    json.dump(syllabi_by_sem, f)

In [107]:
def none_pittsburgh(sections):
    return all(x in ["W", "W1", "W2", "X", "SV"] for x in sections)

In [157]:
# Generate data format for TurtleBot
data_for_tb = {}

SHOW_INFO = False

for sem in syllabi_by_sem.keys():
    if sem[0] not in ["S", "F"]:
        print("Skipping", sem)
        continue
    
    print()
    print("Processing", sem)
    for course, dat in list(syllabi_by_sem[sem].items()):
        s_nonunique, s_other_campuses = get_sections_clean(sem, course)
        s_sections = get_unique_sections(sem, course)
        
        info = []
        if len(s_sections) == 0:
            ss = [x[1] for x in dat]
            if not none_pittsburgh(ss) and not len(s_other_campuses):
                print("Warning: zero sections in SOC", course, sem, ss)
            continue

        assert len(dat) > 0
        coursename = dat[0][2]
        for x in dat:
            match = [y for y in s_sections if _clean_sec_name(y["section"]) == x[1]]
            if len(match):
                inst = match[0]["instructors"]
                if inst == "Instructor TBA" or inst == "TBA":
                    inst = "Unknown Instructor"
                info.append([sem, "{}-{} ({})".format(course, x[1], inst), x[4]])

        if len(info) == 0:
            if SHOW_INFO:
                print("Info: uniqueness failed", course, sem, 
                          [x[1] for x in dat], [x["section"] for x in s_nonunique],
                          [x["section"] for x in s_sections])

            success = False
            for x in dat:
                match = [y for y in s_nonunique if _clean_sec_name(y["section"]) == x[1]]
                if len(match):
                    inst = match[0]["instructors"]
                    if inst == "Instructor TBA" or inst == "TBA":
                        inst = "Unknown Instructor"
                    info.append([sem, "{}-{} ({})".format(course, x[1], inst), x[4]])
                    success = True
                    break

            if not success and \
                    not none_pittsburgh([x[1] for x in dat]) and \
                    not all(x[1] in s_other_campuses for x in dat):

                print("Warning: zero matching sections", course, sem, 
                      [x[1] for x in dat], [x["section"] for x in s_nonunique],
                      [x["section"] for x in s_sections],
                      s_other_campuses)
                continue

        if course not in data_for_tb:
            data_for_tb[course] = ["", []]

        if len(coursename) > len(data_for_tb[course][0]):
            data_for_tb[course][0] = coursename

        data_for_tb[course][1] += info

        #print([x for x in s_sections])
        #print(info)
        #print()


Processing F22
Skipping N22
Skipping M22

Processing S22

Processing F21
Skipping N21
Skipping M21

Processing S21

Processing F20
Skipping N20
Skipping M20

Processing S20

Processing F19
Skipping N19
Skipping M19


In [158]:
with open("data_for_tb.json", "w+") as f:
    json.dump(data_for_tb, f)

In [159]:
len(data_for_tb["76101"][1])

82

In [160]:
data_for_tb["21228"]

['Discrete Mathematics',
 [['S22',
   '21228-1 (Bohman)',
   'https://canvas.cmu.edu/courses/26871/modules/items/5074002'],
  ['S21',
   '21228-1 (Loh)',
   'https://canvas.cmu.edu/courses/20319/modules/items/4735512'],
  ['S21',
   '21228-A (Uy)',
   'https://canvas.cmu.edu/courses/20319/modules/items/4735513'],
  ['S21',
   '21228-B (Chen)',
   'https://canvas.cmu.edu/courses/20319/modules/items/4735514'],
  ['S21',
   '21228-C (Hathcock)',
   'https://canvas.cmu.edu/courses/20319/modules/items/4735515'],
  ['S20',
   '21228-1 (Bohman)',
   'https://canvas.cmu.edu/courses/12937/modules/items/4413170'],
  ['S20',
   '21228-A (Cox)',
   'https://canvas.cmu.edu/courses/12937/modules/items/4413171'],
  ['S20',
   '21228-B (Cox)',
   'https://canvas.cmu.edu/courses/12937/modules/items/4413172'],
  ['S20',
   '21228-C (He)',
   'https://canvas.cmu.edu/courses/12937/modules/items/4413173']]]